# 🦜 LangChain 全栈开发实战

本 Notebook 展示 LangChain 框架的核心技能，包括：
- Prompt Engineering 技术
- 核心组件使用
- 调试监控技术
- 自定义 Tool 和 Agent
- 向量数据库和 RAG
- 对话历史管理


## 📦 环境准备

### 安装说明

下面的安装脚本会：
1. 🔧 清理可能冲突的旧版本包
2. 📦 固定 `protobuf` 版本为 4.25.8（避免冲突）
3. 📦 按顺序安装所有依赖
4. ✅ 确保没有版本冲突

**运行后请重启 Kernel！**

> 💡 **提示**：这个脚本会卸载并重装部分包，可能需要几分钟。如果你已经安装过相关包，建议先创建虚拟环境。


In [ ]:
import subprocess
import sys

def install_dependencies():
    """清理并重新安装所有依赖，解决版本冲突"""
    
    print("🔧 步骤 1: 卸载冲突包...")
    conflict_packages = ["protobuf", "chromadb"]
    for pkg in conflict_packages:
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "-q", pkg], 
                      capture_output=True, check=False)
    print("  ✅ 清理完成\n")
    
    print("📦 步骤 2: 固定 protobuf 版本...")
    subprocess.run([sys.executable, "-m", "pip", "install", "protobuf==4.25.8"])
    print("  ✅ protobuf 4.25.8\n")
    
    print("📦 步骤 3: 安装 LangChain...")
    result = subprocess.run([sys.executable, "-m", "pip", "install", 
                            "langchain", "langchain-openai", "langchain-community", 
                            "tiktoken", "-q"])
    print("  ✅ LangChain 核心包\n")
    
    print("📦 步骤 4: 安装 ChromaDB (保持 protobuf 版本)...")
    subprocess.run([sys.executable, "-m", "pip", "install", 
                   "chromadb", "--no-deps", "-q"])
    print("  安装 ChromaDB 依赖...")
    subprocess.run([sys.executable, "-m", "pip", "install", 
                   "onnxruntime", "tokenizers", "pypika", "tqdm", 
                   "overrides", "importlib-resources", "grpcio>=1.58.0",
                   "bcrypt>=4.0.1", "typer>=0.9.0", "kubernetes>=28.1.0",
                   "tenacity>=8.2.3", "PyYAML>=6.0.0", "posthog>=2.4.0", "-q"],
                  capture_output=True)
    print("  ✅ ChromaDB\n")
    
    print("📦 步骤 5: 安装 API 和工具...")
    subprocess.run([sys.executable, "-m", "pip", "install",
                   "fastapi", "uvicorn", "langserve", 
                   "requests", "beautifulsoup4", "pydantic", "-q"])
    print("  ✅ 服务和工具\n")
    
    print("📦 步骤 6: 安装本地 Embedding 模型...")
    subprocess.run([sys.executable, "-m", "pip", "install",
                   "sentence-transformers", "-q"])
    print("  ✅ Sentence Transformers (用于免费的本地 embeddings)\n")
    
    print("="*60)
    print("✅ 安装完成！")
    print("="*60)
    print("\n💡 下一步: 重启 Kernel (Kernel -> Restart Kernel)\n")

install_dependencies()


🔧 步骤 1: 卸载冲突包...
  ✅ 清理完成

📦 步骤 2: 固定 protobuf 版本...
  Using cached protobuf-4.25.8-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
Using cached protobuf-4.25.8-cp37-abi3-macosx_10_9_universal2.whl (394 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.


  ✅ protobuf 4.25.8

📦 步骤 3: 安装 LangChain...
  ✅ LangChain 核心包

📦 步骤 4: 安装 ChromaDB (保持 protobuf 版本)...
  安装 ChromaDB 依赖...
  ✅ ChromaDB

📦 步骤 5: 安装 API 和工具...
  ✅ 服务和工具

✅ 安装完成！

💡 下一步: 重启 Kernel (Kernel -> Restart Kernel)



In [7]:
import os
from datetime import datetime
from typing import List, Dict, Any, Optional
import json

print("🔧 配置 DeepSeek API")
print("="*60)
print("本 Notebook 使用 DeepSeek 模型（兼容 OpenAI API）")
print("\n📝 获取 API Key:")
print("1. 访问 https://platform.deepseek.com/")
print("2. 注册/登录账号")
print("3. 创建 API Key")
print("4. 将下方的 'your-deepseek-api-key-here' 替换为你的 API Key")
print("\n💰 费用说明:")
print("DeepSeek 比 OpenAI 便宜很多，适合学习和开发")
print("="*60)

os.environ["OPENAI_API_KEY"] = "sk-22fd815ca17a48f08d788e5a5dd454a1"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

print("\n✅ 配置完成！使用模型: deepseek-chat")


🔧 配置 DeepSeek API
本 Notebook 使用 DeepSeek 模型（兼容 OpenAI API）

📝 获取 API Key:
1. 访问 https://platform.deepseek.com/
2. 注册/登录账号
3. 创建 API Key
4. 将下方的 'your-deepseek-api-key-here' 替换为你的 API Key

💰 费用说明:
DeepSeek 比 OpenAI 便宜很多，适合学习和开发

✅ 配置完成！使用模型: deepseek-chat


### 📖 DeepSeek 使用说明

**为什么选择 DeepSeek？**
- 💰 **价格优势**：比 OpenAI 便宜约 90%，性价比极高
- 🇨🇳 **中文友好**：对中文支持优秀，理解能力强
- 🔌 **API 兼容**：完全兼容 OpenAI API 格式，无需修改代码
- 🚀 **性能优秀**：deepseek-chat 模型性能接近 GPT-3.5

**DeepSeek API 配置方式：**
```python
# 方式1: 环境变量（推荐）
os.environ["OPENAI_API_KEY"] = "your-deepseek-api-key"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

# 方式2: 直接在模型中配置
llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_key="your-deepseek-api-key",
    openai_api_base="https://api.deepseek.com"
)
```

**⚠️ 重要说明：Embeddings 方案**

DeepSeek **不支持 Embeddings API**，因此本 Notebook 使用以下方案：

1. **聊天对话**：使用 DeepSeek（便宜快速）
2. **向量化**：使用本地免费模型 `sentence-transformers`

这是最佳组合：
- ✅ 完全免费的 Embeddings
- ✅ 低成本的 LLM 对话
- ✅ 无需多个 API Key

**注意事项：**
1. ⚠️ 记得替换 DeepSeek API Key！
2. 📦 首次运行会自动下载 Embedding 模型（约 500MB）
3. 📊 查看用量：https://platform.deepseek.com/usage


## 一、Prompt Engineering 技术

展示各种提示工程技术的应用


### 1.1 Zero-shot Prompting


In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    temperature=0, 
    model="deepseek-chat",
    openai_api_base="https://api.deepseek.com"
)

zero_shot_prompt = ChatPromptTemplate.from_template(
    "将以下文本分类为正面、负面或中性情感：\n\n{text}"
)

chain = zero_shot_prompt | llm

result = chain.invoke({"text": "这个产品真的很棒，我非常喜欢！"})
print("Zero-shot 结果:", result.content)


Zero-shot 结果: 正面


### 1.2 Few-shot Prompting


In [9]:
from langchain.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "这个产品质量很好", "output": "正面"},
    {"input": "服务态度太差了", "output": "负面"},
    {"input": "价格还可以", "output": "中性"},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个情感分类专家。"),
    few_shot_prompt,
    ("human", "{input}"),
])

chain = final_prompt | llm
result = chain.invoke({"input": "物流速度挺快的"})
print("Few-shot 结果:", result.content)


Few-shot 结果: 正面


### 1.3 Chain of Thought (CoT)


In [10]:
cot_prompt = ChatPromptTemplate.from_template(
    """问题：{question}

让我们一步一步思考：
1. 首先，理解问题的关键信息
2. 然后，列出解决步骤
3. 最后，得出答案

请按照上述步骤回答。"""
)

chain = cot_prompt | llm
result = chain.invoke({"question": "如果一个苹果3元，买5个苹果打8折，需要多少钱？"})
print("CoT 结果:\n", result.content)


CoT 结果:
 好的，我们按步骤来推理。  

---

**1. 理解问题关键信息**  
- 一个苹果原价：3 元  
- 买的数量：5 个  
- 折扣：8 折（即原价的 80%）  

---

**2. 列出解决步骤**  
- 第一步：计算原价总金额  
\[
5 \times 3 = 15 \text{ 元}
\]  
- 第二步：计算打折后的价格  
\[
15 \times 0.8 = 12 \text{ 元}
\]  

---

**3. 得出答案**  
\[
\boxed{12}
\]  

所以，买 5 个苹果打 8 折后需要 **12 元**。


## 二、LangChain 核心组件使用


In [11]:
from langchain_openai import OpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="人物姓名")
    age: int = Field(description="人物年龄")
    occupation: str = Field(description="职业")

parser = PydanticOutputParser(pydantic_object=Person)

prompt = ChatPromptTemplate.from_template(
    """提取以下文本中的人物信息：
{text}

{format_instructions}"""
)

chain = prompt | llm | parser

result = chain.invoke({
    "text": "张三今年30岁，是一名软件工程师",
    "format_instructions": parser.get_format_instructions()
})

print("解析结果:", result)
print(f"姓名: {result.name}, 年龄: {result.age}, 职业: {result.occupation}")


解析结果: name='张三' age=30 occupation='软件工程师'
姓名: 张三, 年龄: 30, 职业: 软件工程师


## 三、Chat History - 对话历史管理


In [12]:
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

print("对话 1:")
print(conversation.predict(input="我喜欢Python编程"))
print("\n" + "="*50 + "\n")

print("对话 2:")
print(conversation.predict(input="我刚才说我喜欢什么？"))

file_history = FileChatMessageHistory("chat_history.json")
file_history.add_user_message("什么是LangChain？")
file_history.add_ai_message("LangChain是一个用于开发LLM应用的框架。")
print("\n对话历史已持久化到文件")


/var/folders/y_/g595hp191h75fwdk_s4cf6bm0000gn/T/ipykernel_88133/2492734910.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/var/folders/y_/g595hp191h75fwdk_s4cf6bm0000gn/T/ipykernel_88133/2492734910.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


对话 1:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 我喜欢Python编程
AI:

> Finished chain.
太棒了！Python是一门非常强大且多用途的编程语言，我很高兴你对它感兴趣！无论是数据分析、人工智能、网络开发还是自动化脚本，Python都能大显身手。你喜欢用Python做什么呢？是开发网站、进行数据分析，还是探索机器学习领域？如果你有任何问题或需要学习资源，我都很乐意帮助你！ 😊


对话 2:


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 我喜欢Python编程
AI: 太棒了！Python是一门非常强大且多用途的编程语言，我很高兴你对它感兴趣！无论是数据分析、人工智能、网络开发还是自动化脚本，Python都能大显身手。你喜欢用Python做什么呢？是开发网站、进行数据分析，还是探索机器学习领域？如果你有任何问题或需要学习资源，

## 四、自定义 Tools 和 Agents


In [13]:
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import MessagesPlaceholder
import requests

@tool
def get_current_time(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """获取当前时间"""
    return datetime.now().strftime(format)

@tool
def calculator(expression: str) -> str:
    """计算数学表达式"""
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"计算错误: {str(e)}"

@tool
def search_web(query: str) -> str:
    """模拟网页搜索"""
    return f"搜索结果：关于 '{query}' 的信息...(这是模拟结果)"

@tool
async def async_search(query: str) -> str:
    """异步搜索工具"""
    import asyncio
    await asyncio.sleep(0.5)
    return f"异步搜索结果: {query}"

print("工具测试:")
print("时间:", get_current_time.invoke({}))
print("计算:", calculator.invoke({"expression": "(10 + 5) * 2"}))
print("搜索:", search_web.invoke({"query": "LangChain教程"}))


工具测试:
时间: 2025-10-12 21:01:28
计算: 30
搜索: 搜索结果：关于 'LangChain教程' 的信息...(这是模拟结果)


In [14]:
tools = [get_current_time, calculator, search_web]

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的AI助手。使用提供的工具来回答问题。"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({
    "input": "现在几点？然后帮我计算 25 * 4 等于多少"
})
print("\nAgent 结果:", result["output"])




> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`
responded: 我来帮您获取当前时间并计算数学表达式。

2025-10-12 21:01:52
Invoking: `calculator` with `{'expression': '25 * 4'}`


100根据查询结果：
- 当前时间是：2025年10月12日 21:01:52
- 25 × 4 = 100

> Finished chain.

Agent 结果: 根据查询结果：
- 当前时间是：2025年10月12日 21:01:52
- 25 × 4 = 100


## 五、向量数据库和 RAG


In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

print("📦 初始化本地 Embedding 模型...")
print("💡 提示：DeepSeek 不支持 Embeddings API，使用免费的本地模型")
print("⏳ 首次运行会下载模型，请稍等...")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ Embedding 模型加载完成！\n")

documents = [
    Document(page_content="LangChain是一个用于开发语言模型应用的框架。", metadata={"source": "doc1"}),
    Document(page_content="向量数据库用于存储和检索向量表示的文档。", metadata={"source": "doc2"}),
    Document(page_content="RAG是检索增强生成，结合检索和生成的技术。", metadata={"source": "doc3"}),
    Document(page_content="Embeddings将文本转换为数值向量表示。", metadata={"source": "doc4"}),
    Document(page_content="Chroma是一个轻量级的向量数据库。", metadata={"source": "doc5"}),
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)

splits = text_splitter.split_documents(documents)

print("🔄 创建向量存储...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print(f"✅ 创建了包含 {len(splits)} 个文档块的向量存储")


📦 初始化本地 Embedding 模型...
💡 提示：DeepSeek 不支持 Embeddings API，使用免费的本地模型
⏳ 首次运行会下载模型，请稍等...


model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding 模型加载完成！

🔄 创建向量存储...
✅ 创建了包含 5 个文档块的向量存储


### 5.1 相似度搜索


In [18]:
query = "什么是RAG？"
results = vectorstore.similarity_search(query, k=2)

print(f"查询: {query}")
print("\n相似度搜索结果:")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

results_with_scores = vectorstore.similarity_search_with_score(query, k=2)
print("\n带相似度分数的结果:")
for doc, score in results_with_scores:
    print(f"分数: {score:.4f} - {doc.page_content}")


查询: 什么是RAG？

相似度搜索结果:

结果 1:
内容: RAG是检索增强生成，结合检索和生成的技术。
元数据: {'source': 'doc3'}

结果 2:
内容: LangChain是一个用于开发语言模型应用的框架。
元数据: {'source': 'doc1'}

带相似度分数的结果:
分数: 1.1480 - RAG是检索增强生成，结合检索和生成的技术。
分数: 1.5999 - LangChain是一个用于开发语言模型应用的框架。


### 5.2 RAG 检索链


In [19]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

template = """基于以下上下文回答问题：

上下文: {context}

问题: {question}

答案:"""

QA_PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_PROMPT},
    return_source_documents=True
)

question = "LangChain是什么？"
result = qa_chain.invoke({"query": question})

print(f"问题: {question}")
print(f"\n答案: {result['result']}")
print("\n来源文档:")
for doc in result['source_documents']:
    print(f"- {doc.page_content}")


问题: LangChain是什么？

答案: 根据上下文，LangChain是一个用于开发语言模型应用的框架。

来源文档:
- LangChain是一个用于开发语言模型应用的框架。
- RAG是检索增强生成，结合检索和生成的技术。
- Chroma是一个轻量级的向量数据库。


### 5.3 带记忆的 RAG 系统


In [20]:
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

question1 = "什么是向量数据库？"
result1 = conversational_chain.invoke({"question": question1})
print(f"Q1: {question1}")
print(f"A1: {result1['answer']}\n")

question2 = "它的主要用途是什么？"
result2 = conversational_chain.invoke({"question": question2})
print(f"Q2: {question2}")
print(f"A2: {result2['answer']}")


Q1: 什么是向量数据库？
A1: 向量数据库是一种专门用于存储和检索向量形式数据的数据库系统。它通过将文档、图像或其他类型的数据转换为数值向量（通常由机器学习模型生成），并利用相似度计算（如余弦相似度）来高效检索与查询内容最相关的信息。例如，Chroma 就是一个轻量级的向量数据库，常用于支持检索增强生成（RAG）等应用场景。

Q2: 它的主要用途是什么？
A2: 根据提供的上下文，向量数据库的主要用途是**存储和检索向量表示的文档**。

简单来说，它就像一个专门为“向量”这种数据格式设计的图书馆，可以高效地存储文档的数学表示（即向量），并根据查询快速找到最相关的结果。


## 六、实战案例：完整的 RAG 问答机器人


In [21]:
class RAGChatBot:
    def __init__(self, llm, vectorstore):
        self.llm = llm
        self.vectorstore = vectorstore
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )
        
        self.retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 3}
        )
        
        self.chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True
        )
    
    def chat(self, question: str) -> Dict[str, Any]:
        result = self.chain.invoke({"question": question})
        return {
            "answer": result["answer"],
            "sources": [doc.page_content for doc in result["source_documents"]]
        }
    
    def get_history(self):
        return self.memory.load_memory_variables({})

chatbot = RAGChatBot(llm, vectorstore)

response1 = chatbot.chat("什么是LangChain？")
print("Q: 什么是LangChain？")
print(f"A: {response1['answer']}")
print(f"\n来源: {response1['sources']}")

print("\n" + "="*50 + "\n")

response2 = chatbot.chat("它和向量数据库有什么关系？")
print("Q: 它和向量数据库有什么关系？")
print(f"A: {response2['answer']}")




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
LangChain是一个用于开发语言模型应用的框架。

RAG是检索增强生成，结合检索和生成的技术。

Chroma是一个轻量级的向量数据库。
Human: 什么是LangChain？

> Finished chain.

> Finished chain.
Q: 什么是LangChain？
A: 根据提供的上下文，LangChain是一个用于开发语言模型应用的框架。

来源: ['LangChain是一个用于开发语言模型应用的框架。', 'RAG是检索增强生成，结合检索和生成的技术。', 'Chroma是一个轻量级的向量数据库。']




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 什么是LangChain？
Assistant: 根据提供的上下文，LangChain是一个用于开发语言模型应用的框架。
Follow Up Input: 它和向量数据库有什么关系？
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain cha

## 七、LangServe - API 服务化


In [22]:
server_code = '''
import os
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langserve import add_routes

os.environ["OPENAI_API_KEY"] = "your-deepseek-api-key-here"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple API server using LangChain with DeepSeek",
)

llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_base="https://api.deepseek.com"
)

prompt = ChatPromptTemplate.from_template(
    "告诉我一个关于{topic}的故事"
)

chain = prompt | llm

add_routes(
    app,
    chain,
    path="/story",
)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)
'''

with open("./scripts/langserve_app.py", "w", encoding="utf-8") as f:
    f.write(server_code)

print("✅ LangServe 服务代码已保存到 scripts/langserve_app.py")
print("   (已配置 DeepSeek 模型)")
print("\n运行方法:")
print("cd scripts && python langserve_app.py")
print("\n访问文档: http://localhost:8000/docs")


✅ LangServe 服务代码已保存到 scripts/langserve_app.py
   (已配置 DeepSeek 模型)

运行方法:
cd scripts && python langserve_app.py

访问文档: http://localhost:8000/docs


## 八、Streamlit Web 界面


In [23]:
streamlit_app = '''
import os
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

os.environ["OPENAI_API_KEY"] = "your-deepseek-api-key-here"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

st.title("🤖 RAG 问答机器人 (DeepSeek)")

if "messages" not in st.session_state:
    st.session_state.messages = []

@st.cache_resource
def load_chain():
    llm = ChatOpenAI(
        temperature=0,
        model="deepseek-chat",
        openai_api_base="https://api.deepseek.com"
    )
    embeddings = OpenAIEmbeddings(
        openai_api_base="https://api.deepseek.com"
    )
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )
    
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )
    
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory,
        return_source_documents=True
    )
    
    return chain

chain = load_chain()

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("请输入您的问题..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    with st.chat_message("assistant"):
        with st.spinner("思考中..."):
            response = chain.invoke({"question": prompt})
            answer = response["answer"]
            st.markdown(answer)
            
            with st.expander("查看来源文档"):
                for i, doc in enumerate(response["source_documents"]):
                    st.markdown(f"**来源 {i+1}:** {doc.page_content}")
    
    st.session_state.messages.append({"role": "assistant", "content": answer})

if st.sidebar.button("清除对话历史"):
    st.session_state.messages = []
    st.rerun()
'''

with open("./scripts/streamlit_app.py", "w", encoding="utf-8") as f:
    f.write(streamlit_app)

print("✅ Streamlit 应用已保存到 scripts/streamlit_app.py")
print("   (已配置 DeepSeek 模型)")
print("\n运行方法:")
print("streamlit run scripts/streamlit_app.py")


✅ Streamlit 应用已保存到 scripts/streamlit_app.py
   (已配置 DeepSeek 模型)

运行方法:
streamlit run scripts/streamlit_app.py


## 九、总结

本 Notebook 全面展示了 LangChain 框架的核心开发技能：

### ✅ 已掌握的技能

1. **Prompt Engineering 技术**
   - Zero-shot、Few-shot、COT、ReAct、Prompt Chaining

2. **核心组件使用**
   - LLM、Chat Models、PromptTemplates、Output Parsers、Chains (LCEL)

3. **调试监控**
   - Verbose 日志、Debug 模式、自定义回调处理器

4. **对话历史管理**
   - 内存管理、文件持久化、带记忆的对话链

5. **Tools 和 Agents**
   - 自定义工具、同步/异步调用、Agent 执行器
   - 天气查询、计算器、网页搜索等实用工具

6. **向量数据库和 RAG**
   - Chroma 向量存储、相似度搜索、RAG 检索链
   - 带记忆的 RAG 系统

7. **文档处理**
   - 文档加载器、文档分割器、文本预处理

8. **服务部署**
   - LangServe API 服务、Streamlit Web 界面

### 🚀 扩展方向

- 多模态支持（图像、音频）
- 流式输出优化
- 高级 RAG 技术（重排序、混合检索）
- LangSmith 深度集成
- 生产环境优化

### 💡 最佳实践

- 使用 LCEL 构建可组合的链
- 合理使用 Verbose 和 Debug 进行调试
- 为生产环境配置适当的内存管理策略
- 使用向量数据库优化检索性能
- 实现错误处理和重试机制
- 监控 Token 使用和成本

---

**🎉 恭喜！你已经掌握了 LangChain 的核心开发技能！**
